# Making our agents Modular

A DSPy signature defines what are the inputs and outputs of your task is. Though a Module defines how that task is executed. Lets see an example of this so we can see the pro's of using Modules

In [0]:
%pip install dspy mlflow
dbutils.library.restartPython()

In [0]:
import dspy
from typing import Literal
import mlflow

llama = dspy.LM('databricks/databricks-meta-llama-3-3-70b-instruct', cache=False)
dspy.configure(lm = llama)

mlflow.dspy.autolog()

## Lets bring back our Review and Support Signatures

In [0]:
class AnalyzeReview(dspy.Signature):
    """Analyze a product review to determine sentiment, extract key features, and provide a summary."""

    review_text : str = dspy.InputField()
    product_category : str = dspy.InputField()

    sentiment : Literal["positive", "negative", "neutral"] = dspy.OutputField() # We can enforce types
    key_features = dspy.OutputField()
    summary = dspy.OutputField(desc="A concise summary of the review content.")    #If we need to guide output field, we can add a description

class GenerateSupportResponse(dspy.Signature):
    """Generate a customer support response based on review analysis."""

    sentiment = dspy.InputField(desc="The overall sentiment from the review analysis.")
    key_features = dspy.InputField(format=list, desc="Key features extracted from the review.")
    summary = dspy.InputField(desc="Summary of the review content.")

    support_response = dspy.OutputField(desc="A customer support response addressing the review.")


If you remember before, to use these two signatures together, we had to call predictors on them individually. By wrapping them up in a module, we can call this module to excute the logic. This is essentially the same as standard programming, but our class is the chained logic

In [0]:
class ReviewSupportModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze = dspy.Predict(AnalyzeReview)
        self.respond = dspy.Predict(GenerateSupportResponse)


    def forward(self, review_text, product_category):  #Forward method is the entry point for the module
        analysis = self.analyze(review_text=review_text, product_category=product_category)
        response = self.respond(
            sentiment=analysis.sentiment,
            key_features=analysis.key_features,
            summary=analysis.summary
        )
        return dspy.Prediction(support_response=response.support_response)

In [0]:
support_module = ReviewSupportModule()

review_text = "I love this vacuum, however I wish the bag size was bigger."
product_category = "Electronics"

result = support_module(review_text=review_text, product_category=product_category)

print(result.support_response)


In [0]:
class ReviewSupportModulev2(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze = dspy.Predict(AnalyzeReview)
        self.respond = dspy.Predict(GenerateSupportResponse)

    def add_prefix(self, key_features):
        return "category: " + key_features


    def forward(self, review_text, product_category):  #Forward method is the entry point for the module
        analysis = self.analyze(review_text=review_text, product_category=product_category)
        analysis.key_features = self.add_prefix(analysis.key_features)
        response = self.respond(
            sentiment=analysis.sentiment,
            key_features=analysis.key_features,
            summary=analysis.summary
        )
        return dspy.Prediction(support_response=response.support_response)